In [307]:
import os
import sys
import boto3
from botocore.client import Config
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import psycopg2

# S3 Configuration
ACCESS_KEY_ID = 'YOUR_ACCESS_KEY_ID'
ACCESS_SECRET_KEY = 'YOUR_ACCESS_SECRET_KEY'
BUCKET_NAME = 'YOUR_BUCKET_NAME'
# --------------------------------------------------------------
# The path of the folder(local) where we save the images--(change only the folderpath)
folderpath = r"C:/PATH OF THE FOLDER FOR IMAGES" # make sure to put the 'r' in front
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
all_files = []
# Names of the images inside the folder
fileKeys=os.listdir(folderpath)
# --------------------------------------------------

# S3 Connect
s3 = boto3.resource(
    's3',
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=ACCESS_SECRET_KEY,
    config=Config(signature_version='s3v4')
)
# Function that saves the images in S3 Account and makes them public
def saveS3(fileKey,data):
    s3.Bucket(BUCKET_NAME).put_object(Key=fileKey, Body=data)
    ss=s3.Bucket(BUCKET_NAME).Object(fileKey).Acl().put(ACL = 'public-read')
# ----------------------------------------------------------
def get_exif_data(image):
    """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
    exif_data = {}
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for t in value:
                    sub_decoded = GPSTAGS.get(t, t)
                    gps_data[sub_decoded] = value[t]

                    exif_data[decoded] = gps_data
            else:
                    exif_data[decoded] = value

    return exif_data
# ----------------------------
def _get_if_exist(data, key):
    if key in data:
        return data[key]
		
    return None
# -------------------------------------------------------------------
# Gets the GPSInfo tag from metadata off the images
def get_lat_lon(exif_data):
    """Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)"""
    lat = None
    lon = None

    if "GPSInfo" in exif_data:		
        gps_info = exif_data["GPSInfo"]
        gps_latitude = _get_if_exist(gps_info, "GPSLatitude")  
        gps_latitude_ref = _get_if_exist(gps_info, 'GPSLatitudeRef')
        gps_longitude = _get_if_exist(gps_info, 'GPSLongitude')
        gps_longitude_ref = _get_if_exist(gps_info, 'GPSLongitudeRef')
    return gps_latitude, gps_longitude
# ------------------------------------
# Functions for converting Lat and Lon from dms to decimal
def Latitude_converting(lat):
    deg=float(lat[0])
    minute=float(lat[1])/60
    sec=float(lat[2])/3600
    return deg+minute+sec
# ---------------------------
def Lontitude_coverting(lon):
    deg=float(lon[0])
    minute=float(lon[1])/60
    sec=float(lon[2])/3600
    return deg+minute+sec
# ---------------------------------
# Python3 code to remove whitespace 
def remove(string): 
    return string.replace(" ", "") 
# ------------------------------------------------------------------------------------
# connection to database and inserting the data --the records must be an array of  values 
def bulkInsert(records):
    try:
#         parameters for connecting to Postgres Db (change only the params)
        connection = psycopg2.connect(user="username",
                                      password="pasword",
                                      host="127.0.0.1",
                                      port="5432",
                                      database="Databaze_Name")
        cursor = connection.cursor()
        sql_insert_query = """ INSERT INTO table_name (title,geom,url) 
                           VALUES (%s,ST_GeomFromText(%s,3815),%s) """

        # executemany() to insert multiple rows rows
        result = cursor.executemany(sql_insert_query, records)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into Your_table_name table")

    except (Exception, psycopg2.Error) as error:
        print("Failed inserting record into Your_table_name table {}".format(error))

    finally:
        # closing database connection.
        if (connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")            
#  ---------------------------------------------------------------------------------------------           
# Loop that prints metadata of each image,Lat and Lon of each off them, takes the name of them 
for i in range(0, len(filepaths),1):
#  """names and paths of the images """
    path=filepaths[i]
    name=fileKeys[i]
    with open(path,'rb') as f:
#  """gets the metadata of images,GPSInfo tag --> Coordinates Lat Lon """
        file = f.read()
        data = open(path, 'rb')
        images = Image.open(path)
        saveS3(name,data);
        exifdatas = get_exif_data(images)
#         lat and lon
        latlongitude=get_lat_lon(exifdatas)
# -----------------------------------------------
# Lat and lon sepperated and converted to decimal
        lat=latlongitude[0]
        lon=latlongitude[1]
        lat_c=Latitude_converting(lat)   
        lon_c=Lontitude_coverting(lon)
# ------------------------------------------------
# Parameters modified for saving in Database
        coordinates = 'POINT(%s %s)' % (lat_c,lon_c)
        slice_jpg=slice(0,-4)
        name_db= remove(name[slice_jpg])
        name_no_space=remove(name)
#        -- change the region --
        url='https://amazon-version1.s3.eu-central-1.amazonaws.com/{0}'.format(name)
        records_to_insert=[(name_db,coordinates,url)]
#         Saves the array of data to DB
        bulkInsert(records_to_insert)
    


                        
                         



1 Record inserted successfully into poi table
PostgreSQL connection is closed
1 Record inserted successfully into poi table
PostgreSQL connection is closed
1 Record inserted successfully into poi table
PostgreSQL connection is closed
1 Record inserted successfully into poi table
PostgreSQL connection is closed
1 Record inserted successfully into poi table
PostgreSQL connection is closed
